In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
import nltk
# Launch the installer to download "gutenberg" and "stop words" corpora.
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
# Import the data we just downloaded and installed.
from nltk.corpus import gutenberg, stopwords

# Grab and process the raw data.
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [4]:
moby = gutenberg.raw('melville-moby_dick.txt')

In [6]:
print (moby[0:200])

[Moby Dick by Herman Melville 1851]


ETYMOLOGY.

(Supplied by a Late Consumptive Usher to a Grammar School)

The pale Usher--threadbare in coat, heart, body, and brain; I see him
now.  He was


In [13]:
# This pattern matches all text between square brackets.
pattern = "[\[].*?[\]]"
moby_noTitle = re.sub(pattern, "", moby)

print(moby_noTitle[0:200])




ETYMOLOGY.

(Supplied by a Late Consumptive Usher to a Grammar School)

The pale Usher--threadbare in coat, heart, body, and brain; I see him
now.  He was ever dusting his old lexicons and 


In [14]:
# Now we'll match and remove chapter headings.
moby_noChaps = re.sub(r'CHAPTER .*', '', moby_noTitle)

In [16]:
# Remove newlines and other extra whitespace by splitting and rejoining.
moby_noWhite = ' '.join(moby_noChaps.split())

print(moby_noWhite[0:100])

ETYMOLOGY. (Supplied by a Late Consumptive Usher to a Grammar School) The pale Usher--threadbare in 


In [19]:
import spacy
nlp = spacy.load('en')

# All the processing work is done here, so it may take a while.
moby_doc = nlp(moby_noWhite[0:999999])

In [20]:
# Let's explore the objects we've built.
print("The moby_doc object is a {} object.".format(type(moby_doc)))
print("It is {} tokens long".format(len(moby_doc)))
print("The first three tokens are '{}'".format(moby_doc[:3]))
print("The type of each token is {}".format(type(moby_doc[0])))

The moby_doc object is a <class 'spacy.tokens.doc.Doc'> object.
It is 212262 tokens long
The first three tokens are 'ETYMOLOGY. ('
The type of each token is <class 'spacy.tokens.token.Token'>


In [21]:
from collections import Counter

# Utility function to calculate how frequently words appear in the text.
def word_frequencies(text, include_stop=True):
    
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            words.append(token.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(words)

In [23]:
moby_freq = word_frequencies(moby_doc, include_stop=False).most_common(10)
print(moby_freq)


[('I', 1694), ("'s", 1283), ('whale', 804), ('But', 584), ('like', 503), ('The', 470), ('ship', 407), ('man', 384), ('sea', 330), ('old', 319)]


In [24]:
# Utility function to calculate how frequently lemas appear in the text.
def lemma_frequencies(text, include_stop=True):
    
    # Build a list of lemas.
    # Strip out punctuation and, optionally, stop words.
    lemmas = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            lemmas.append(token.lemma_)
            
    # Build and return a Counter object containing word counts.
    return Counter(lemmas)

In [25]:
moby_lemma_freq = lemma_frequencies(moby_doc, include_stop=False).most_common(10)
print('moby:', moby_lemma_freq)

moby: [('-PRON-', 2477), ('whale', 1331), ("'s", 968), ('but', 584), ('man', 576), ('the', 541), ('like', 527), ('ship', 503), ('be', 466), ('sea', 416)]


In [31]:
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

moby_lemma = [ lmtzr.lemmatize(token) for token in moby_doc]


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/christophersmyth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


AttributeError: 'spacy.tokens.token.Token' object has no attribute 'endswith'

In [34]:
import nltk
from nltk.corpus import wordnet

lmtzr = nltk.WordNetLemmatizer()
word = "Here are words, and cars."
tokens = nltk.word_tokenize(word)
token_lemma = [ lmtzr.lemmatize(token) for token in tokens ]
tagged = nltk.pos_tag(token_lemma)
chunking = nltk.chunk.ne_chunk(tagged)